In [1]:
import sys
import os
SCRIPT_DIR = os.path.dirname(os.path.abspath("."))
sys.path.append(SCRIPT_DIR)
import helper
from utils import data_utils
import matplotlib.pyplot as plt
from utils import training_utils
from utils import data_utils
import torch
from model import models
import json
import os
from model import lightning_models
import math
import torchvision

/home/richard/miniconda3/envs/dl_env/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.23). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [6]:
model_dir = "../best_training_imagenet"
pretrained_model_path = os.path.join(model_dir,"ssl",'ssl-epoch=999.ckpt')
save_model_path = os.path.join(model_dir,"ssl",'last_epoch_backbone_resnet50.ckpt')
# if loading from GPU
device = torch.device("cuda:0")

In [7]:
config = helper.Config(model_dir,default_config_file="../default_configs/default_config_imagenet1k.ini")
if "CIFAR" in config.DATA["dataset"] or "MNIST" in config.DATA["dataset"]:
    prune_backbone = True
else:
    prune_backbone = False
ssl_model = lightning_models.CLAP(backbone_name = config.SSL["backbone"],
                                  prune = prune_backbone,
                                  use_projection_head=config.SSL["use_projection_head"],
                                  proj_dim = config.SSL["proj_dim"],
                                  proj_out_dim = config.SSL["proj_out_dim"],
                                  loss_name= config.SSL["loss_function"],
                                  optim_name = config.SSL["optimizer"],
                                  lr = 1.0,
                                  scheduler_name = config.SSL["lr_scheduler"],
                                  momentum = config.SSL["momentum"],
                                  weight_decay = config.SSL["weight_decay"],
                                  eta = config.SSL["lars_eta"],
                                  warmup_epochs = config.SSL["warmup_epochs"],
                                  n_epochs = config.SSL["n_epochs"],
                                  n_views = config.DATA["n_views"],
                                  batch_size = config.SSL["batch_size"],
                                  lw0 = config.SSL["lw0"],
                                  lw1 = config.SSL["lw1"],
                                  lw2 = config.SSL["lw2"],
                                  rs = config.SSL["rs"],
                                  pot_pow = config.SSL["pot_pow"])

Loading default settings...
[SemiSL]does not exist in the config file
[TL]does not exist in the config file
[KNN]does not exist in the config file
[SemiSL]does not exist in the config file
[TL]does not exist in the config file
[KNN]does not exist in the config file
[INFO]
num_nodes = 4
gpus_per_node = 4
cpus_per_gpu = 12
prefetch_factor = 2
precision = 16-mixed
fix_random_seed = True
strategy = ddp
if_profile = False

[DATA]
dataset = IMAGENET1K
n_views = 4
n_trans = 2
augmentation_package = albumentations
augmentations = ['RandomResizedCrop', 'GaussianBlur', 'RandomGrayscale', 'ColorJitter', 'RandomHorizontalFlip', 'RandomSolarize']
crop_size = [224]
crop_min_scale = [0.08]
crop_max_scale = [1.0]
hflip_prob = [0.5]
blur_kernel_size = [23]
blur_prob = [0.5]
grayscale_prob = [0.2]
jitter_brightness = [0.8]
jitter_contrast = [0.8]
jitter_saturation = [0.8]
jitter_hue = [0.2]
jitter_prob = [0.8]
solarize_prob = [0.0]
imagenet_train_dir = ../datasets/imagenet1k/train/train.lmdb
imagenet_va

In [10]:
if os.path.isfile(pretrained_model_path):
    print(f'Found pretrained model at {pretrained_model_path}, loading...')
    ssl_model = lightning_models.CLAP.load_from_checkpoint(pretrained_model_path)
else:
    assert "Model not found!"
torch.save(ssl_model.backbone.net.state_dict(),save_model_path)

Found pretrained model at ../best_training_imagenet/ssl/ssl-epoch=999.ckpt, loading...
max_mem_size is dummy for RepulsiveEllipsoidPackingLossUnitNorm
lw2 is dummy for RepulsiveEllipsoidPackingLossUnitNorm


In [11]:
#test if the model can be loaded properly
model = torchvision.models.resnet50()
model.fc = torch.nn.Identity()
model.load_state_dict(torch.load(save_model_path,weights_only=True))

<All keys matched successfully>

In [16]:
for name,module in model.named_modules():
    if name:
        print(name)
    

conv1
bn1
relu
maxpool
layer1
layer1.0
layer1.0.conv1
layer1.0.bn1
layer1.0.conv2
layer1.0.bn2
layer1.0.conv3
layer1.0.bn3
layer1.0.relu
layer1.0.downsample
layer1.0.downsample.0
layer1.0.downsample.1
layer1.1
layer1.1.conv1
layer1.1.bn1
layer1.1.conv2
layer1.1.bn2
layer1.1.conv3
layer1.1.bn3
layer1.1.relu
layer1.2
layer1.2.conv1
layer1.2.bn1
layer1.2.conv2
layer1.2.bn2
layer1.2.conv3
layer1.2.bn3
layer1.2.relu
layer2
layer2.0
layer2.0.conv1
layer2.0.bn1
layer2.0.conv2
layer2.0.bn2
layer2.0.conv3
layer2.0.bn3
layer2.0.relu
layer2.0.downsample
layer2.0.downsample.0
layer2.0.downsample.1
layer2.1
layer2.1.conv1
layer2.1.bn1
layer2.1.conv2
layer2.1.bn2
layer2.1.conv3
layer2.1.bn3
layer2.1.relu
layer2.2
layer2.2.conv1
layer2.2.bn1
layer2.2.conv2
layer2.2.bn2
layer2.2.conv3
layer2.2.bn3
layer2.2.relu
layer2.3
layer2.3.conv1
layer2.3.bn1
layer2.3.conv2
layer2.3.bn2
layer2.3.conv3
layer2.3.bn3
layer2.3.relu
layer3
layer3.0
layer3.0.conv1
layer3.0.bn1
layer3.0.conv2
layer3.0.bn2
layer3.0.conv3